In [10]:
# Bibliotecas
import pandas as pd
import numpy as np

In [11]:
# Carregamento dos "Dados Antigos"
df_antigos = pd.read_csv('data/processos_2022.csv')

# Amostra
df_antigos.head()

,processo,data_distribuicao,data_baixa,entrancia,comarca,serventia,nome_area_acao,is_segredo_justica,codg_classe,codg_assuntos
0,0119071.75.2004.8.09.0051,2022-05-25,2022-06-30,FINAL,GOIÂNIA,2ª Vara Cível,upj civel,False,7,10671
1,0168391.94.2004.8.09.0051,2022-05-20,2022-05-20,FINAL,GOIÂNIA,3ª Vara Cível,upj civel,False,7,10671
2,0189657.40.2004.8.09.0051,2022-06-02,2024-01-22,FINAL,GOIÂNIA,31ª Vara Cível,upj civel,False,7,10671
3,0197944.89.2004.8.09.0051,2022-06-07,2022-10-07,FINAL,GOIÂNIA,22ª Vara Cível,upj civel,False,7,10671
4,0211274.56.2004.8.09.0051,2022-06-09,2022-08-03,FINAL,GOIÂNIA,8ª Vara Cível,upj civel,False,7,10671


In [12]:
# Cópia do dataframe
df = df_antigos.copy()

In [13]:
# Limpeza e tratamento
# Verificar o nome correto das colunas (pode haver diferenças de acentuação ou espaços)
colunas = df.columns.tolist()

# Encontrar as colunas de data corretamente
coluna_serventia = [col for col in colunas if 'serventia' in col.lower()][0]
coluna_distribuicao = [col for col in colunas if 'data_distribuicao' in col.lower()][0]
coluna_baixa = [col for col in colunas if 'data_baixa' in col.lower()][0]
coluna_area_acao = [col for col in colunas if 'nome_area_acao' in col.lower()][0]
coluna_processo_id = [col for col in colunas if 'processo' in col.lower()][0]
coluna_comarca = [col for col in colunas if 'comarca' in col.lower()][0]

# Renomear colunas para garantir consistência
df = df.rename(columns={
coluna_distribuicao: 'data_distribuicao',
coluna_baixa: 'data_baixa',
coluna_area_acao: 'nome_area_acao',
coluna_processo_id: 'processo',
coluna_comarca: 'comarca',
coluna_serventia: 'serventia'
})

# Converter colunas de data para datetime com tratamento de erros
df['data_distribuicao'] = pd.to_datetime(df['data_distribuicao'], errors='coerce')
df['data_baixa'] = pd.to_datetime(df['data_baixa'], errors='coerce')

In [19]:
# CRIAÇÃO DAS ESTATÍSTICAS POR ANO ('comarca', 'serventia', 'nome_area_acao'):

# 1. Extrair ano de distribuição e baixa
df['ano_distribuicao'] = df['data_distribuicao'].dt.year
df['ano_baixa'] = df['data_baixa'].dt.year

# Chave de agrupamento para as estatísticas
grouping_keys = ['comarca', 'serventia', 'nome_area_acao']

# 2. Calcular Distribuídos por ano
distribuidos_df = df.dropna(subset=['ano_distribuicao']).groupby(
    ['ano_distribuicao'] + grouping_keys
).size().reset_index(name='Distribuídos')
distribuidos_df = distribuidos_df.rename(columns={'ano_distribuicao': 'ano'})

# 3. Calcular Baixados por ano
baixados_df = df.dropna(subset=['ano_baixa']).groupby(
    ['ano_baixa'] + grouping_keys
).size().reset_index(name='Baixados')
baixados_df = baixados_df.rename(columns={'ano_baixa': 'ano'})

# 4. Calcular Pendentes por ano de distribuição
pendentes_df = df[df['data_baixa'].isna()].dropna(subset=['ano_distribuicao']).groupby(
    ['ano_distribuicao'] + grouping_keys
).size().reset_index(name='Pendentes')
pendentes_df = pendentes_df.rename(columns={'ano_distribuicao': 'ano'})

# 5. Juntar tudo em uma única tabela (estatisticas)
estatisticas = pd.merge(distribuidos_df, baixados_df, on=['ano'] + grouping_keys, how='outer')
estatisticas = pd.merge(estatisticas, pendentes_df, on=['ano'] + grouping_keys, how='outer')

# 6. Limpar e organizar o DataFrame final
estatisticas = estatisticas.fillna(0)
estatisticas[['Distribuídos', 'Baixados', 'Pendentes']] = estatisticas[['Distribuídos', 'Baixados', 'Pendentes']].astype(int)

# Remover linhas onde o ano não pôde ser determinado
estatisticas = estatisticas.dropna(subset=['ano'])
estatisticas['ano'] = estatisticas['ano'].astype(int)

# 7. Calcular a Taxa de Congestionamento por ano
soma_pend_baix = estatisticas['Pendentes'] + estatisticas['Baixados']
estatisticas['Taxa de Congestionamento (%)'] = np.where(
    soma_pend_baix > 0,
    (estatisticas['Pendentes'] / soma_pend_baix) * 100,
    0
).round(2)

# 8. Ordenar e reordenar colunas para melhor visualização
estatisticas = estatisticas.sort_values(by=['ano', 'comarca', 'serventia', 'nome_area_acao'], ascending=[False, True, True, True])

colunas_finais = [
    'ano', 'nome_area_acao', 'comarca', 'serventia', 
    'Distribuídos', 'Baixados', 'Pendentes', 'Taxa de Congestionamento (%)'
]
estatisticas = estatisticas[colunas_finais]

estatisticas.sample(10)

,ano,nome_area_acao,comarca,serventia,Distribuídos,Baixados,Pendentes,Taxa de Congestionamento (%)
3678,2024,juizado especial fazenda publica,GOIANÉSIA,Vara das Fazendas Públicas e Registros Públicos,0,58,0,0.0
2404,2023,juizado especial civel,ITABERAÍ,"1ª Vara Cível, Infância e da Juventude e Juiza...",0,382,0,0.0
5846,2025,criminal,SANTA CRUZ DE GOIÁS,Vara Judicial,0,8,0,0.0
4404,2024,civel,SANTA HELENA DE GOIÁS,"2ª Vara (Cív., Criminal – crime em geral, crim...",0,61,0,0.0
845,2022,fazenda publica mista,ITAJÁ,Vara Judicial,139,12,20,62.5
6024,2025,orgao especial,TRIBUNAL DE JUSTIÇA,GABINETE DESA NELMA BRANCO FERREIRA PERILO,0,1,0,0.0
2787,2023,civel,PORANGATU,"2ª Vara Cível, das Fazendas Públicas e de Regi...",0,92,0,0.0
3092,2023,camara civel,TRIBUNAL DE JUSTIÇA,GABINETE DESA. ROBERTA NASSER LEONE,0,353,0,0.0
1713,2023,juizado especial civel,ANICUNS,"1ª Vara Judicial (Família e Sucessões, Infânci...",0,456,0,0.0
2776,2023,fazenda publica mista,PONTALINA,Vara Judicial,0,73,0,0.0


In [20]:
# CRIAÇÃO DAS ESTATÍSTICAS POR MÊS ('comarca', 'serventia', 'nome_area_acao'):

# 1. Extrair mês de distribuição e baixa
df['mes_distribuicao'] = df['data_distribuicao'].dt.month
df['mes_baixa'] = df['data_baixa'].dt.month

# Chave de agrupamento para as estatísticas
grouping_keys = ['comarca', 'serventia', 'nome_area_acao']

# 2. Calcular Distribuídos por mês
distribuidos_mes_df = df.dropna(subset=['ano_distribuicao', 'mes_distribuicao']).groupby(
    ['ano_distribuicao', 'mes_distribuicao'] + grouping_keys
).size().reset_index(name='Distribuídos_mes')
distribuidos_mes_df = distribuidos_mes_df.rename(columns={'ano_distribuicao': 'ano', 'mes_distribuicao': 'mes'})

# 3. Calcular Baixados por mês
baixados_mes_df = df.dropna(subset=['ano_baixa', 'mes_baixa']).groupby(
    ['ano_baixa', 'mes_baixa'] + grouping_keys
).size().reset_index(name='Baixados_mes')
baixados_mes_df = baixados_mes_df.rename(columns={'ano_baixa': 'ano', 'mes_baixa': 'mes'})

# 4. Calcular Pendentes por mês de distribuição
pendentes_mes_df = df[df['data_baixa'].isna()].dropna(subset=['ano_distribuicao', 'mes_distribuicao']).groupby(
    ['ano_distribuicao', 'mes_distribuicao'] + grouping_keys
).size().reset_index(name='Pendentes_mes')
pendentes_mes_df = pendentes_mes_df.rename(columns={'ano_distribuicao': 'ano', 'mes_distribuicao': 'mes'})

# 5. Juntar tudo em uma única tabela (estatisticas_mes)
merge_keys = ['ano', 'mes'] + grouping_keys
estatisticas_mes = pd.merge(distribuidos_mes_df, baixados_mes_df, on=merge_keys, how='outer')
estatisticas_mes = pd.merge(estatisticas_mes, pendentes_mes_df, on=merge_keys, how='outer')

# 6. Limpar e organizar o DataFrame final
estatisticas_mes = estatisticas_mes.fillna(0)
estatisticas_mes[['Distribuídos_mes', 'Baixados_mes', 'Pendentes_mes']] = estatisticas_mes[['Distribuídos_mes', 'Baixados_mes', 'Pendentes_mes']].astype(int)

# Remover linhas onde o mês não pôde ser determinado
estatisticas_mes = estatisticas_mes.dropna(subset=['ano', 'mes'])
estatisticas_mes[['ano', 'mes']] = estatisticas_mes[['ano', 'mes']].astype(int)

# 7. Calcular a Taxa de Congestionamento por mês
soma_pend_baix_mes = estatisticas_mes['Pendentes_mes'] + estatisticas_mes['Baixados_mes']
estatisticas_mes['Taxa de Congestionamento_mes (%)'] = np.where(
    soma_pend_baix_mes > 0,
    (estatisticas_mes['Pendentes_mes'] / soma_pend_baix_mes) * 100,
    0
).round(2)

# 8. Ordenar e reordenar colunas para melhor visualização
estatisticas_mes = estatisticas_mes.sort_values(by=['ano', 'mes','comarca', 'serventia', 'nome_area_acao'], ascending=[False, True, True, True, True])

colunas_finais_mes = [
    'ano', 'mes', 'nome_area_acao', 'comarca', 'serventia', 
    'Distribuídos_mes', 'Baixados_mes', 'Pendentes_mes', 'Taxa de Congestionamento_mes (%)'
]
estatisticas_mes = estatisticas_mes[colunas_finais_mes]

estatisticas_mes.sample(10)

,ano,mes,nome_area_acao,comarca,serventia,Distribuídos_mes,Baixados_mes,Pendentes_mes,Taxa de Congestionamento_mes (%)
17284,2022,12,fazenda publica mista,TAQUARAL DE GOIÁS,Vara Judicial,11,0,2,100.00
31588,2024,1,juizado especial civel,ALVORADA DO NORTE,Vara Judicial,0,5,0,0.00
46010,2025,2,civel,QUIRINÓPOLIS,"2ª Vara (Cível, das Faz. Púb., de Reg. Púb. e ...",0,21,0,0.00
7517,2022,6,juizado especial civel,CAMPOS BELOS,"1ª Vara Judicial (Família e Sucessões, Infânci...",9,7,0,0.00
4184,2022,3,camara civel,TRIBUNAL DE JUSTIÇA,GABINETE DES. ITAMAR DE LIMA,112,27,1,3.57
22927,2023,5,juizado especial civel,JANDAIA,Vara Judicial,0,24,0,0.00
52091,2025,8,infancia e juventude civel,QUIRINÓPOLIS,1ª Vara (Cível e da Inf. e da Juv.),0,1,0,0.00
1018,2022,1,infancia e juventude civel,PIRENÓPOLIS,"1ª Vara Judicial (Família e Sucessões, Infânci...",1,0,0,0.00
27607,2023,9,juizado especial civel,IACIARA,Vara Judicial,0,2,0,0.00
17508,2023,1,juizado especial criminal,ACREÚNA,"2ª Vara Judicial (Fazendas Públicas, Criminal,...",0,1,0,0.00
